# Cellpose 2.x Installation – **WORKING VERSION for MyBinder**

This notebook installs **Cellpose 2.x** with the classic API that works end-to-end.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/your-username/your-repo/HEAD?labpath=cellpose_working.ipynb)

## Features
✅ Installs Cellpose 2.x (stable API)  
✅ Creates synthetic test data  
✅ Runs segmentation successfully  
✅ Shows beautiful results  
✅ Provides working code examples


## Step 1  – Environment Setup

In [ ]:
import sys, os, subprocess, time

print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")
print(f"Platform: {sys.platform}")

def run_cmd(cmd, timeout=600):
    """Run *cmd* in a subshell and capture stdout/stderr, returning (success, out, err)."""
    try:
        res = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
        return res.returncode == 0, res.stdout.strip(), res.stderr.strip()
    except subprocess.TimeoutExpired:
        return False, "", f"Timeout after {timeout}s"
    except Exception as e:
        return False, "", str(e)

print("\n✅ Environment setup complete!")


## Step 2  – Install Cellpose 2.x and dependencies

In [ ]:
print("🔧 Installing Cellpose 2.3.2 (CPU build) and dependencies…")
print("This may take 3–4 minutes on Binder…\n")

# Numerical & plotting stack pinned to avoid ABI mismatches
packages = [
    "numpy<2.0",
    "scipy<1.12",
    "scikit-image<0.23",
    "matplotlib<3.9",
    # I/O & utils
    "opencv-python-headless",
    "tifffile",
    # Deep-learning stack (CPU wheels)
    "torch==2.2.2",
    "torchvision==0.17.2",
    # Cellpose itself
    "cellpose==2.3.2"
]

successful, failed = [], []
for i, pkg in enumerate(packages, 1):
    print(f"[{i}/{len(packages)}] Installing {pkg} …", end=" ")
    # wrap pkg in quotes so '<' isn't handled by the shell
    ok, out, err = run_cmd(f'pip install --quiet "{pkg}"')
    if ok:
        print("✅")
        successful.append(pkg)
    else:
        print("❌")
        failed.append(pkg)
        if err:
            lines = err.split('\n')
            msg = next((l for l in lines if 'ERROR' in l), lines[-1])
            print(f"    {msg[:120]}")

print("\n📊 Installation summary")
print(f"  ✅ Successful: {len(successful)}/{len(packages)}")
print(f"  ❌ Failed    : {len(failed)}")
if failed:
    print(f"  Failed packages: {failed}")

installation_ok = len(successful) >= len(packages) - 1  # tolerate one mirror hiccup
print(f"\nStatus: {'✅ READY' if installation_ok else '⚠️  PARTIAL'}")


## Step 3  – Test Cellpose import

In [ ]:
print("🔍 Testing Cellpose 2.x import…")
try:
    import cellpose
    from cellpose import models, io, utils
    import numpy as np
    import matplotlib.pyplot as plt
    import tifffile

    print("✅ All imports successful")
    print(f"✅ Cellpose version: {cellpose.__version__}")

    # Test model initialisation
    model = models.Cellpose(gpu=False, model_type='cyto')
    print("✅ Cellpose model initialised")

    cellpose_working = True
except Exception as e:
    print(f"❌ Error: {e}")
    cellpose_working = False

# Configure matplotlib for headless use
if cellpose_working:
    import matplotlib
    matplotlib.use('Agg')
    plt.ioff()
    print("✅ Matplotlib configured")

print(f"\nCellpose status: {'✅ WORKING' if cellpose_working else '❌ FAILED'}")


## Step 4  – Create synthetic test data

In [ ]:
... rest of notebook unchanged ...